### Assigment Description

Pada tugas kali ini, kita akan melihat bagaimana cara untuk menggunakan query dengan gaya SQL pada Python dengan memanfaatkan beberapa library yang digunakan untuk melakukan beberapa *workflow* dibawah ini:

Adapun workflow untuk tugas ini adalah sebagai berikut:
1. Pembuatan inisial script yang kemudian akan memanfaatkan memori sementara untuk menyimpan hasil *data seeding* yang ada.
2. Hasil penyimpanan kemudian di write ke dalam data berbentuk csv.
3. Melakukan *data querying* dengan gaya sql dari data csv.

### Import Libraries

In [4]:
!pip install duckdb

import pandas as pd
import duckdb
import sqlite3
import csv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Sqlite3 and CSV




`Sqlite3` digunakan untuk dapat melakukan pembuatan database sementara pada memori untuk kemudian kita manfaatkan untuk melakukan pembuatan tabel seperti *database* pada umumnya. Setelah pembuatan tabel dilakukan module `csv` akan dimanfaatkan untuk dapat melakukan penulisan data yang sudah ada dalam memori menjadi suatu data csv untuk dapat kemudian kita operasikan seperti layaknya *dataframe* pada umumnya.

### Duckdb

`DuckDB` adalah library eksternal yang digunakan untuk SQL pada Python, namun tidak secara spesifik terintegrasi dengan Pandas. `DuckDB` adalah suatu alat atau *tools* untuk mengelola sistem *database* yang dilakukan secara *in-memory* yang dirancang untuk mengeksekusi kueri SQL secara efisien untuk keperluan analitik. `DuckDB` menyediakan interface untuk bahasa pemrograman Python untuk dapat mengeksekusi kueri SQL dengan skrip Python atau dalam suatu notebook.

untuk dokumentasi yang lebih lengkap dapat dilihat [disini](https://duckdb.org/docs/api/python/overview.html) (Duckdb Python API)

#### Usage Example

In [5]:
# Sample DataFrame (also can be read from csv formatted data)
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston']
}

# Pay attention to the name of the dataframe defined here because that also will be used in the query.
df = pd.DataFrame(data)
df

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago
3,David,40,Houston


In [6]:
# Query definition
example_query = """
SELECT * FROM df
WHERE Age > 30
"""

# Dataframe result
example_result = duckdb.query(example_query).df()
example_result

,Name,Age,City
0,Charlie,35,Chicago
1,David,40,Houston


In [7]:
# example_result is already a ready to used pandas dataframe
type(example_result)

pandas.core.frame.DataFrame

### Workflow 1: Initial Script Table Creation Query Question

#### Query 1:
Buatlah query untuk membuat table `Customers` dengan spesifikasi sebagai berikut:

> 1. `CustomerID` berupa suatu integer.
>
> 2. `FirstName` berupa suatu string dengan panjang karakter maksimal 50.
>
> 3. `LastName` berupa suatu string dengan panjang karakter maksimal 50.
>
> 4. `Email` berupa suatu string dengan panjang karakter maksimal 100.
>
> 5. `Phone` berupa suatu string dengan panjang karakter maksimal 15.
>
>**Primary Key**: `CustomerID`



In [22]:
customers_table_creation_query = """
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    Phone VARCHAR(15)
);
"""

#### Query 2:
Buatlah query untuk membuat table `Products` dengan spesifikasi sebagai berikut:

> 1. `ProductID` berupa suatu string dengan panjang karakter maksimal 10.
>
> 2. `ProductName` berupa suatu string dengan panjang karakter maksimal 50.
>
> 3. `Price` berupa suatu string dengan panjang karakter maksimal 25.
>
>**Primary Key**: `ProductID`


In [23]:
products_table_creation_query = """
CREATE TABLE Products (
    ProductID VARCHAR(10) PRIMARY KEY,
    ProductName VARCHAR(50),
    Price VARCHAR(25)
);
"""

#### Query 3:
Buatlah query untuk membuat table `Orders` dengan spesifikasi sebagai berikut:
> 1. `OrderID` berupa suatu integer
>
> 2. `CustomerID` berupa suatu integer.
>
> 3. `ProductID` berupa suatu integer.
>
> 4. `Quantity` berupa suatu integer.
>
> 5. `OrderDate` berupa suatu timestamp.
>
>**Primary Key**: `OrderID`
>
>**Foreign Key**:
> * `CustomerID` menuju `CustomerID` yang ada pada table `Customers`.
> * `ProductID` menuju `ProductID` yang ada pada table `Products`

In [26]:
orders_table_creation_query = """
CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    ProductID VARCHAR(10),
    Quantity INT,
    OrderDate TIMESTAMP,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
"""

#### Data Seeding

Secara umum, data seeding adalah proses pengisian data awal ke dalam suatu basis data (database) agar aplikasi atau sistem yang menggunakan basis data tersebut dapat berfungsi dengan baik saat pertama kali dijalankan. Tujuan dari data seeding adalah untuk menyediakan data dasar yang diperlukan untuk menjalankan aplikasi atau sistem secara minimal, sehingga pengguna dapat langsung memulai penggunaan aplikasi tanpa perlu menambahkan data secara manual.

Pada tugas kali ini, data seeding digunakan untuk mengisi *database* secara spesifik pada tiap tabel yang telah dibuat sebelumnya.

In [41]:
# Connect to an in-memory SQLite database
# conn = sqlite3.connect(':memory:')
conn = duckdb.connect(':memory:')
cursor = conn.cursor()

# Define SQL statements
sql_commands = f"""
{customers_table_creation_query}
{products_table_creation_query}
{orders_table_creation_query}

INSERT INTO Products (ProductID, ProductName, Price) VALUES
('P131', 'Wireless Keyboard', 'Rp55.000,00'),
('P132', 'Noise-Canceling Headphones', 'Rp120.000,00'),
('P133', 'Smart Home Camera', 'Rp180.000,00'),
('P134', 'Wireless Charging Stand', 'Rp65.000,00'),
('P135', 'Portable SSD', 'Rp220.000,00'),
('P136', 'Smart Doorbell', 'Rp150.000,00'),
('P137', 'Ergonomic Chair', 'Rp400.000,00'),
('P138', 'LED Desk Lamp', 'Rp40.000,00'),
('P139', 'Coffee Grinder', 'Rp90.000,00'),
('P140', 'Bluetooth Speaker Set', 'Rp130.000,00'),
('P141', 'External DVD Drive', 'Rp75.000,00'),
('P142', 'Fitness Smartwatch', 'Rp180.000,00'),
('P143', 'Desktop Printer', 'Rp300.000,00'),
('P144', 'Gaming Mouse', 'Rp70.000,00'),
('P145', 'Cordless Vacuum Cleaner', 'Rp250.000,00'),
('P146', 'Wireless Earbuds', 'Rp50.000,00'),
('P147', 'Mesh Office Chair', 'Rp450.000,00'),
('P148', 'Smart Scale', 'Rp60.000,00'),
('P149', 'Portable Projector', 'Rp320.000,00'),
('P150', 'Wireless Gaming Controller', 'Rp100.000,00'),
('P151', 'Bluetooth Keyboard', 'Rp55.000,00'),
('P152', 'External Blu-ray Drive', 'Rp90.000,00'),
('P153', 'Smart Mirror', 'Rp280.000,00'),
('P154', 'Office Desk Organizer', 'Rp25.000,00'),
('P155', 'USB Desk Fan', 'Rp35.000,00'),
('P156', 'Smart Light Bulbs', 'Rp85.000,00'),
('P157', 'Portable Air Cooler', 'Rp120.000,00'),
('P158', 'Gaming Headset', 'Rp110.000,00'),
('P159', 'Wireless Mouse Pad Charger', 'Rp45.000,00'),
('P160', 'Smart WiFi Plug', 'Rp30.000,00');

INSERT INTO Customers (CustomerID, FirstName, LastName, Email, Phone) VALUES
(1, 'John', 'Smith', 'john.smith@email.com', '123-456-7890'),
(2, 'Emily', 'Johnson', 'emily.j@email.com', '987-654-3210'),
(3, 'Mike', 'Davis', 'mike.d@email.com', '555-123-4567'),
(4, 'Sarah', 'Wilson', 'sarah.w@email.com', '333-555-7777'),
(5, 'David', 'Lee', 'david.lee@email.com', '777-888-9999'),
(6, 'Lisa', 'Anderson', 'lisa.anderson@email.com', '444-666-1111'),
(7, 'Brian', 'Taylor', 'brian.taylor@email.com', '222-333-4444'),
(8, 'Emma', 'White', 'emma.white@email.com', '888-999-0000'),
(9, 'Jason', 'Miller', 'jason.miller@email.com', '555-444-3333'),
(10, 'Olivia', 'Brown', 'olivia.brown@email.com', '111-222-3333'),
(11, 'Daniel', 'Martinez', 'daniel.martinez@email.com', '555-123-4567'),
(12, 'Jessica', 'Turner', 'jessica.turner@email.com', '777-888-9999'),
(13, 'Matthew', 'Harris', 'matthew.harris@email.com', '444-666-1111'),
(14, 'Sophia', 'Clark', 'sophia.clark@email.com', '222-333-4444'),
(15, 'Ethan', 'Garcia', 'ethan.garcia@email.com', '888-999-0000'),
(16, 'Isabella', 'Allen', 'isabella.allen@email.com', '555-444-3333'),
(17, 'Noah', 'Taylor', 'noah.taylor@email.com', '111-222-3333'),
(18, 'Mia', 'King', 'mia.king@email.com', '555-123-4567'),
(19, 'Aiden', 'Turner', 'aiden.turner@email.com', '777-888-9999'),
(20, 'Zoey', 'Hill', 'zoey.hill@email.com', '444-666-1111'),
(21, 'Elijah', 'Butler', 'elijah.butler@email.com', '222-333-4444'),
(22, 'Ava', 'Powell', 'ava.powell@email.com', '888-999-0000'),
(23, 'Jackson', 'Murphy', 'jackson.murphy@email.com', '555-444-3333'),
(24, 'Scarlett', 'Brooks', 'scarlett.brooks@email.com', '111-222-3333'),
(25, 'Logan', 'Hughes', 'logan.hughes@email.com', '555-123-4567'),
(26, 'Sophie', 'Baker', 'sophie.baker@email.com', '555-123-7890'),
(27, 'Nathan', 'Young', 'nathan.y@email.com', '987-654-3210'),
(28, 'Emily', 'Wilson', 'emily.w@email.com', '555-123-4567');

INSERT INTO Orders (OrderID, CustomerID, ProductID, Quantity, OrderDate) VALUES
(621, 11, 'P131', 2, '2024-05-06 09:30:00'),
(622, 12, 'P132', 1, '2024-05-06 12:45:00'),
(623, 13, 'P133', 3, '2024-05-07 14:15:00'),
(624, 14, 'P134', 1, '2024-05-07 17:30:00'),
(625, 15, 'P135', 2, '2024-05-08 10:30:00'),
(626, 16, 'P136', 1, '2024-05-08 13:45:00'),
(627, 17, 'P137', 4, '2024-05-09 15:30:00'),
(628, 18, 'P138', 2, '2024-05-09 18:00:00'),
(629, 19, 'P139', 1, '2024-05-10 11:15:00'),
(630, 20, 'P140', 3, '2024-05-10 14:45:00'),
(631, 11, 'P141', 2, '2024-05-21 10:30:00'),
(632, 12, 'P142', 1, '2024-05-21 12:45:00'),
(633, 13, 'P143', 3, '2024-05-22 14:15:00'),
(634, 14, 'P144', 1, '2024-05-22 17:30:00'),
(635, 15, 'P145', 2, '2024-05-23 10:30:00'),
(636, 16, 'P146', 1, '2024-05-23 13:45:00'),
(637, 17, 'P147', 4, '2024-05-24 15:30:00'),
(638, 18, 'P148', 2, '2024-05-24 18:00:00'),
(639, 19, 'P149', 1, '2024-05-25 11:15:00'),
(640, 20, 'P150', 3, '2024-05-25 14:45:00'),
(661, 11, 'P131', 2, '2024-06-06 09:30:00'),
(662, 12, 'P132', 1, '2024-06-06 12:45:00'),
(663, 13, 'P133', 3, '2024-06-07 14:15:00'),
(664, 14, 'P134', 1, '2024-06-07 17:30:00'),
(665, 15, 'P135', 2, '2024-06-08 10:30:00'),
(666, 16, 'P136', 1, '2024-06-08 13:45:00'),
(667, 17, 'P137', 4, '2024-06-09 15:30:00'),
(668, 18, 'P138', 2, '2024-06-09 18:00:00'),
(669, 19, 'P139', 1, '2024-06-10 11:15:00'),
(670, 20, 'P140', 3, '2024-06-10 14:45:00'),
(671, 26, 'P131', 2, '2024-06-11 09:30:00'),
(672, 27, 'P132', 1, '2024-06-11 12:45:00'),
(673, 1, 'P150', 2, '2024-07-23 10:30:00'),
(674, 4, 'P146', 1, '2024-07-23 13:45:00'),
(675, 5, 'P148', 4, '2024-07-24 15:30:00'),
(676, 12, 'P154', 2, '2024-07-24 18:00:00'),
(677, 6, 'P157', 1, '2024-07-25 11:15:00'),
(678, 20, 'P150', 3, '2024-07-25 14:45:00'),
(679, 11, 'P144', 2, '2024-07-06 09:30:00'),
(680, 7, 'P135', 1, '2024-07-06 12:45:00'),
(681, 8, 'P153', 3, '2024-07-07 14:15:00'),
(682, 14, 'P158', 1, '2024-07-07 17:30:00'),
(683, 15, 'P160', 2, '2024-08-08 10:30:00'),
(684, 16, 'P146', 1, '2024-08-08 13:45:00'),
(685, 17, 'P145', 4, '2024-08-09 15:31:00'),
(686, 18, 'P141', 2, '2024-08-09 18:04:00'),
(687, 2, 'P135', 1, '2024-08-10 11:17:00'),
(688, 6, 'P142', 3, '2024-08-10 14:23:00'),
(689, 26, 'P156', 2, '2024-08-11 09:11:00'),
(690, 14, 'P147', 1, '2024-08-11 12:23:00'),
(691, 12, 'P139', 3, '2024-08-12 14:45:00');
"""

# Execute SQL commands
# cursor.executescript(sql_commands)
conn.execute(sql_commands)

### Workflow 2: Write Database Data to CSV Format

In [42]:
# Fetch data from Customers table
cursor.execute("SELECT * FROM Customers")
customers_data = cursor.fetchall()

# Fetch data from Products table
cursor.execute("SELECT * FROM Products")
products_data = cursor.fetchall()

# Fetch data from Orders table
cursor.execute("SELECT * FROM Orders")
orders_data = cursor.fetchall()

# Write Customers data to CSV
with open('Customers.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(['CustomerID', 'FirstName', 'LastName', 'Email', 'Phone'])
    csv_writer.writerows(customers_data)

# Write Products data to CSV
with open('Products.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(['ProductID', 'ProductName', 'Price'])
    csv_writer.writerows(products_data)

# Write Orders data to CSV
with open('Orders.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(['OrderID', 'CustomerID', 'ProductID', 'Quantity', 'OrderDate'])
    csv_writer.writerows(orders_data)

# Close connection
# conn.close()

print("CSV files created successfully.")

CSV files created successfully.


In [43]:
# TODO: read the previously made Customers.csv file, name it as 'customers' and show the first 5 rows.
customers_df = pd.read_csv('Customers.csv')
customers_df.head()

,CustomerID,FirstName,LastName,Email,Phone
0,1,John,Smith,john.smith@email.com,123-456-7890
1,2,Emily,Johnson,emily.j@email.com,987-654-3210
2,3,Mike,Davis,mike.d@email.com,555-123-4567
3,4,Sarah,Wilson,sarah.w@email.com,333-555-7777
4,5,David,Lee,david.lee@email.com,777-888-9999


In [44]:
# TODO: read the previously made Orders.csv file, name it as 'orders' and show the first 5 rows.
orders_df = pd.read_csv('Orders.csv')
orders_df.head()

,OrderID,CustomerID,ProductID,Quantity,OrderDate
0,621,11,P131,2,2024-05-06 09:30:00
1,622,12,P132,1,2024-05-06 12:45:00
2,623,13,P133,3,2024-05-07 14:15:00
3,624,14,P134,1,2024-05-07 17:30:00
4,625,15,P135,2,2024-05-08 10:30:00


In [45]:
# TODO: read the previously made Products.csv file, name it as 'products' and show the first 5 rows.
products_df = pd.read_csv('Products.csv')
products_df.head()

,ProductID,ProductName,Price
0,P131,Wireless Keyboard,"Rp55.000,00"
1,P132,Noise-Canceling Headphones,"Rp120.000,00"
2,P133,Smart Home Camera,"Rp180.000,00"
3,P134,Wireless Charging Stand,"Rp65.000,00"
4,P135,Portable SSD,"Rp220.000,00"


### Workflow 3: Data Querying on Answering Research Questions

#### Research Question 1:

>Tampilkan 5 produk terbanyak berdasarkan total penjualan diurutkan berdasarkan total penjualan dari terbesar hingga terkecil
>
>Skema yang diharapkan:
* **id_produk**
* **total_harga**

In [54]:
query_q1 = """
SELECT 
    Products.ProductID AS id_produk, 
    SUM(CAST(REPLACE(REPLACE(REPLACE(Price, 'Rp', ''), '.', ''), ',', '.') AS FLOAT) * Quantity) AS total_harga
FROM Products, Orders
WHERE Products.ProductID = Orders.ProductID
GROUP BY Products.ProductID
ORDER BY total_harga DESC
LIMIT 5;
"""

result_q1 = conn.query(query_q1).df()
result_q1

,id_produk,total_harga
0,P137,3200000.0
1,P147,2250000.0
2,P145,1500000.0
3,P135,1320000.0
4,P133,1080000.0


#### Research Question 2:

>Berapa rata-rata jumlah pembelian produk secara umum setiap bulannya?
>
>Skema yang diharapkan:
* **bulan**
* **rata_rata_pemesanan**

In [57]:
query_q2 = """
SELECT
    MONTH(OrderDate) AS bulan,
    AVG(Quantity) AS rata_rata_pemesanan
FROM Orders
GROUP BY MONTH(OrderDate);

"""

result_q2 = conn.query(query_q2).df()
result_q2

,bulan,rata_rata_pemesanan
0,5,2.000000
1,6,1.916667
2,7,2.000000
3,8,2.111111


#### Research Question 3:

Tampilkan 10 pengguna yang memiliki total pesanan terbanyak diurutkan dari total pesanan terbanyak hingga yang paling sedikit.

Skema yang diharapkan:
* **id_pelanggan**
* **nama_pelanggan**: nama lengkap pelanggan (nama awal dan nama akhir).
* **total_pesanan**: total pesanan pelanggan dari keseluruhan data pesanan.

In [61]:
query_q3 = """
SELECT
    C.CustomerID AS id_pelanggan,
    CONCAT(C.FirstName, ' ', C.LastName) AS nama_pelanggan,
    COUNT(O.Quantity) AS total_pesanan
FROM Customers C
JOIN Orders O ON C.CustomerID = O.CustomerID
GROUP BY id_pelanggan, nama_pelanggan
ORDER BY total_pesanan DESC
LIMIT 10;
"""

result_q3 = conn.query(query_q3).df()
result_q3

,id_pelanggan,nama_pelanggan,total_pesanan
0,12,Jessica Turner,5
1,14,Sophia Clark,5
2,17,Noah Taylor,4
3,11,Daniel Martinez,4
4,15,Ethan Garcia,4
5,16,Isabella Allen,4
6,18,Mia King,4
7,20,Zoey Hill,4
8,13,Matthew Harris,3
9,19,Aiden Turner,3


# Theoritical Questions

1. Menurut Anda, mengapa konsep *data warehousing* sangat diperlukan dalam pengolahan data yang ada pada perusahaan modern? (5)

>Jawaban:
> 
> Konsep data warehousing sangat penting dalam pengolahan data pada perusahaan modern karena beberapa alasan utama:
> - **Integrasi Data**: Perusahaan modern sering kali memiliki data yang tersebar di berbagai sumber dan sistem, seperti CRM, ERP, sistem penjualan, media sosial, dan lain-lain. Data warehouse menyediakan platform terpusat untuk mengintegrasikan data ini dari berbagai sumber, memungkinkan perusahaan untuk memiliki pandangan yang konsolidasi dan holistik terhadap operasional dan performa bisnisnya.
> - **Pengambilan Keputusan Berbasis Data**: Dengan mengumpulkan data historis dan saat ini dalam satu tempat, data warehouse memungkinkan analisis mendalam dan pembuatan laporan yang komprehensif. Ini mendukung pengambilan keputusan yang berbasis data, membantu perusahaan untuk merumuskan strategi bisnis yang lebih efektif berdasarkan insight yang diperoleh dari data.
> - **Performa dan Skalabilitas**: Data warehouses dirancang untuk query dan analisis data yang efisien, bahkan dengan volume data yang sangat besar. Ini memastikan bahwa pengguna bisnis dapat mengakses informasi yang mereka butuhkan dengan cepat, tanpa mengganggu operasional sistem transaksional utama. Data warehouse juga dapat diskalakan untuk menampung pertumbuhan data seiring waktu, mendukung kebutuhan bisnis yang berkembang.
> - **Kualitas dan Konsistensi Data**: Data warehousing melibatkan proses ETL (Extract, Transform, Load), yang membersihkan dan mentransformasi data untuk memastikan kualitas dan konsistensi. Ini sangat penting untuk memastikan bahwa analisis dan laporan yang dihasilkan akurat dan dapat diandalkan, mendukung keputusan bisnis yang tepat.
> - **Histori Data dan Analisis Tren**: Data warehouse memungkinkan penyimpanan data historis yang luas, yang tidak praktis untuk disimpan dalam sistem transaksional. Ini memungkinkan perusahaan untuk melakukan analisis tren jangka panjang, mengidentifikasi pola dan peluang, serta memprediksi masa depan dengan model prediktif, memberikan keunggulan kompetitif yang signifikan dalam pasar yang kompetitif.

2. Suatu organisasi yang memiliki skema data yang dikerjakan sebelumnya, perlu melihat jumlah penjualan dan total customer per bulan dari transaksi. Tentukan measure apa saja yang diperlukan? (5)

> Jawaban:
> 
> Untuk melihat jumlah penjualan dan total customer per bulan dari transaksi dalam suatu organisasi, Anda perlu menentukan beberapa measure atau metrik kunci berikut:
> - **Jumlah Penjualan per Bulan (*Total Sales per Month*)**: Ini adalah measure kuantitatif yang merefleksikan total pendapatan atau nilai penjualan yang dihasilkan dalam setiap bulan. Biasanya, ini dihitung dengan menjumlahkan nilai semua transaksi penjualan yang terjadi dalam bulan tersebut. Dalam konteks basis data, ini bisa diwakili oleh sum dari kolom yang menandakan nilai transaksi atau harga produk dikalikan dengan kuantitas yang terjual.
> - **Total Customer per Bulan (*Unique Customers per Month*)**: Metrik ini menghitung jumlah customer unik yang melakukan transaksi dalam setiap bulan. Ini memerlukan identifikasi customer yang unik (misalnya melalui ID customer) dan menghitung berapa banyak customer unik yang ada dalam setiap bulan. Hal ini menunjukkan seberapa luas jangkauan bisnis Anda terhadap customer dalam periode waktu tertentu.
> - **Rata-Rata Nilai Penjualan per Transaksi (*Average Sale Value per Transaction*)**: Meskipun tidak secara langsung diminta, metrik ini sering kali penting untuk dipertimbangkan karena memberikan insight mengenai nilai rata-rata transaksi penjualan. Ini dihitung dengan membagi total penjualan per bulan dengan jumlah transaksi penjualan yang terjadi dalam bulan tersebut.
> - **Jumlah Transaksi per Bulan (*Total Transactions per Month*)**: Ini mengukur berapa banyak transaksi penjualan yang terjadi setiap bulannya. Metrik ini berguna untuk memahami volume aktivitas penjualan, terlepas dari jumlah customer unik atau nilai penjualan total.
> - **Frekuensi Pembelian per Customer (*Purchase Frequency per Customer*)**: Meskipun ini lebih merupakan metrik sekunder, mengetahui seberapa sering customer rata-rata melakukan pembelian dalam periode tertentu (misalnya, per bulan) bisa memberikan wawasan tentang loyalitas dan keterlibatan customer.

3. Bagaimana hubungan antara tabel orders, customers, dan products dapat dimodelkan dalam konteks sebuah data warehouse? (10)
* Apakah Anda akan menggunakan dimensi bintang (star schema), dimensi salju (snowflake schema), atau skema lainnya?
* Mengapa Anda memilih skema tersebut?
* Tentukan peran setiap tabel pada kasus tersebut!

> Jawaban:
>
> Dalam konteks data warehouse untuk hubungan antara Orders, Customers, dan Products, star schema biasanya lebih disukai karena alasan berikut. 
> - **Sederhana dan Intuitif**: Star schema lebih sederhana dan mudah dipahami, membuatnya ideal untuk analisis bisnis dan pembuatan laporan yang efisien.
> - **Performa Query Tinggi**: Struktur star schema yang terdenormalisasi seringkali menghasilkan performa query yang lebih baik karena mengurangi jumlah join yang diperlukan saat melakukan query.
> - **Mendukung Analisis OLAP**: Struktur star schema sangat cocok untuk operasi OLAP (Online Analytical Processing), seperti drilling down, rolling up, dan slicing and dicing.
> Berikut ini peranan masing-masing tabel pada skema bintang.
> - **Tabel Fakta (Orders)**: Tabel Orders berfungsi sebagai tabel fakta yang mencatat metrik-metrik kuantitatif dari setiap transaksi, seperti Quantity dan Total Price. Tabel fakta ini berada di pusat skema bintang dan terhubung langsung ke tabel-tabel dimensi.
> - **Tabel Dimensi (Customers, Products)**:
>   - Tabel Customers berfungsi sebagai tabel dimensi yang menyimpan atribut-atribut yang mendeskripsikan pelanggan, seperti CustomerID, FirstName, LastName, Email, dan Phone. Tabel ini membantu dalam analisis segmen pelanggan, perilaku pembelian, dan demografi pelanggan.
>   - Tabel Products berfungsi sebagai tabel dimensi lain yang menyimpan detail tentang produk, seperti ProductID, ProductName, dan Price. Ini memungkinkan analisis terhadap performa produk, kategori produk populer, dan tren penjualan produk.


4. Bagaimana Peran Metadata dalam Data Warehousing? (5)

> Jawaban:
> - **Mendefinisikan Struktur Data**: Metadata menyediakan definisi tentang tabel, kolom, tipe data, dan hubungan antara berbagai elemen dalam gudang data. Ini membantu dalam pemahaman struktur data yang disimpan dan bagaimana data tersebut diorganisir, yang kritis untuk desain dan implementasi gudang data yang efektif.
> - **Memfasilitasi Integrasi Data: Dalam proses ETL (*Extract, Transform, Load*)**, metadata digunakan untuk memetakan data dari sumber ke gudang data, menentukan bagaimana data diubah atau dibersihkan, dan mengontrol proses loading data. Metadata transformasi memberikan panduan tentang bagaimana data sumber diubah ke format yang sesuai untuk analisis.
> - **Meningkatkan Aksesibilitas dan Pemahaman**: Metadata deskriptif, seperti label, deskripsi, dan dokumentasi tentang data, membantu pengguna bisnis dan analis memahami konten dan konteks data, memudahkan penggunaan data untuk analisis dan pengambilan keputusan.
> - **Mendukung Administrasi dan Manajemen**: Metadata operasional, seperti informasi tentang kapan data dimuat, frekuensi pembaruan, sumber data, dan riwayat perubahan, membantu administrator gudang data dalam pengelolaan, pemeliharaan, dan pemantauan kinerja gudang data.
> - **Menyediakan Kontrol Kualitas dan Konsistensi**: Metadata mengatur aturan dan kebijakan yang berkaitan dengan kualitas data, standar, dan konsistensi, memastikan bahwa data dalam gudang tersebut akurat, dapat diandalkan, dan konsisten di seluruh organisasi.
> - **Mendukung Keamanan dan Kepatuhan**: Metadata keamanan mencakup informasi tentang kontrol akses, peran pengguna, dan kebijakan privasi yang memastikan data sensitif dilindungi dan gudang data mematuhi peraturan seperti GDPR, HIPAA, dll.
> - **Membantu dalam Pemulihan dan Audit Data**: Metadata tentang proses backup, strategi pemulihan, dan log operasi membantu dalam pemulihan data yang efektif dalam kasus kerusakan atau kehilangan data. Ini juga mendukung audit dan pelacakan penggunaan data untuk keperluan kepatuhan dan pengawasan.